In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.applications import InceptionV3


In [9]:
import os
import pandas as pd

dataset_dir = "C:\\Users\\dalil\\Desktop\\Tesis_2025\\Tutorial_yolo_\\Tutorial_yolo\\dataset"  # Ajusta la ruta
csv_output = "dataset_labels.csv"

# Recorrer train, test y val
data = []
for split in ["train", "test", "val"]:
    split_path = os.path.join(dataset_dir, split)
    if os.path.exists(split_path):
        for category in os.listdir(split_path):
            category_path = os.path.join(split_path, category)
            if os.path.isdir(category_path):
                for image_name in os.listdir(category_path):
                    if image_name.lower().endswith((".jpg", ".jpeg", ".png")):
                        image_path = os.path.join(category_path, image_name)
                        data.append((image_path, category, split))

# Guardar en un CSV
df = pd.DataFrame(data, columns=["image_path", "label", "split"])
df.to_csv(csv_output, index=False)

print(f"CSV guardado en {csv_output}")
print(df.head())


CSV guardado en dataset_labels.csv
                                          image_path     label  split
0  C:\Users\dalil\Desktop\Tesis_2025\Tutorial_yol...  fantasia  train
1  C:\Users\dalil\Desktop\Tesis_2025\Tutorial_yol...  fantasia  train
2  C:\Users\dalil\Desktop\Tesis_2025\Tutorial_yol...  fantasia  train
3  C:\Users\dalil\Desktop\Tesis_2025\Tutorial_yol...  fantasia  train
4  C:\Users\dalil\Desktop\Tesis_2025\Tutorial_yol...  fantasia  train


In [ ]:
# Obtener etiquetas únicas y codificarlas numéricamente
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])
num_classes = len(label_encoder.classes_)

# Dividir en train, val y test
train_df = df[df["split"] == "train"]
val_df = df[df["split"] == "val"]
test_df = df[df["split"] == "test"]  # Agregado para testeo

# Función para cargar imágenes
def load_images(df, img_size=(224, 224)):
    images = []
    labels = []
    for _, row in df.iterrows():
        img = cv2.imread(row["image_path"])
        img = cv2.resize(img, img_size)
        img = img.astype("float32") / 255.0  # Normalizar
        images.append(img)
        labels.append(row["label"])
    return np.array(images), to_categorical(labels, num_classes)

# Cargar imágenes
X_train, y_train = load_images(train_df)
X_val, y_val = load_images(val_df)
X_test, y_test = load_images(test_df)  # Agregado para testeo

# Cargar modelo preentrenado EfficientNetB0
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congelar capas preentrenadas

# Construcción del modelo
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=outputs)

# Compilar el modelo
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


Eficiennet

In [ ]:
# Entrenar el modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)


# Evaluar el modelo en el conjunto de testeo
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

# Matriz de confusión y reporte de clasificación
conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred)

# Imprimir resultados
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)


Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.2493 - loss: 1.4252 - val_accuracy: 0.2488 - val_loss: 1.3851
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.2756 - loss: 1.4012 - val_accuracy: 0.2488 - val_loss: 1.3929
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.2483 - loss: 1.4197 - val_accuracy: 0.2488 - val_loss: 1.3920
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.2401 - loss: 1.4158 - val_accuracy: 0.2585 - val_loss: 1.4106
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.2563 - loss: 1.4162 - val_accuracy: 0.2512 - val_loss: 1.4049
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step 
Accuracy: 0.2692
Precision: 0.1294
Recall: 0.2692
F1-score: 0.1262
Confusion Matrix:
 [[ 1  0 49  0]
 [ 0  0 50  0]
 [ 2  0 48  0]
 [ 2  0 30  0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.20      0.02      0.04        50
           1       0.00      0.00      0.00        50
   

c:\Users\dalil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dalil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dalil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo